In [1]:
#LOAD DATA
from dataclasses import dataclass
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display
import plotly.figure_factory as ff

from Quantapp.Universe import Universe
plt.rcParams["figure.figsize"] = (20, 7)

import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from numpy_ext import rolling_apply

from Quantapp.Algorithm   import Algorithm
from Quantapp.Computation import Computation
from Quantapp.Portfolio   import Portfolio
from Quantapp.Plot        import Plot
from Quantapp.DataManager import DataManager
from Quantapp.Universe    import Universe

csv_file_paths = [
    'csv_files/Broad Market.csv',
    'csv_files/S&P 500.csv',
    'csv_files/Russell 1000.csv'
]

dm   = DataManager()
universe  = Universe(file_paths = csv_file_paths)
pm   = Portfolio(universe = universe)
comp = Computation()
plot = Plot()
algorithm = Algorithm()

tickers = pm.add_csv('csv_files/portfolio')


pm.load(period='3y',interval='1d');


benchmark      = dm.retrieve_ticker_data('SPY', period='3y', interval='1d')
risk_free_rate = dm.retrieve_ticker_data('^IRX', period='3y',interval='1d').reindex_like(benchmark)
scalar_multiplier = .7




In [2]:
#Portfolio Details

values = pm.retrieve_sectors()
names = list(values.keys())
values = list(values.values())


details = pm.position_details.copy()
#details['BP Usage %'] = details['BP Usage % ex cash'].apply(lambda s: float(s.replace('%', '')))
sector_details = details.groupby('Sector').sum()

width = 1400
height= 500


pie_chart_layout = make_subplots(rows=1,
                    cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=('Diversity Exposure by Sector','Buying Exposure Percentage by Sector'),
                    vertical_spacing = 0.05)


pie_chart_layout.add_trace(go.Pie(labels=names,values=values),row=1,col=1)

bp_usage  = list(sector_details['BP Usage % ex cash'])
names   = list(sector_details.index)

pie_chart_layout.add_trace(go.Pie(labels=names,values=bp_usage),row=1,col=2)
pie_chart_layout.update_layout(title_text="Exposure")
pie_chart_layout.update_layout(autosize=True)
pie_chart_layout.show(config={'responsive': True})


portfolio_theta = details[['Symbol','Theta']].set_index('Symbol').sort_values('Theta',ascending=False)
portfolio_delta = details[['Symbol','Delta']].set_index('Symbol').sort_values('Delta',ascending=False)
portfolio_gamma = details[['Symbol','Gamma']].set_index('Symbol').sort_values('Gamma',ascending=False)
portfolio_vega = details[['Symbol','Vega']].set_index('Symbol').sort_values('Vega',ascending=False)
portfolio_theta_delta_ratio = details[['Symbol','Theta / Delta']].set_index('Symbol')
portfolio_vega_delta_ratio = details[['Symbol','Vega / Delta']].set_index('Symbol')
portfolio_delta_gamma_ratio = details[['Symbol','Delta / Gamma']].set_index('Symbol')


sector_portfolio_theta = sector_details['Theta']
sector_portfolio_delta = sector_details['Delta']
sector_portfolio_gamma = sector_details['Gamma']
sector_portfolio_vega = sector_details['Vega']
#sector_portfolio_theta_delta_ratio = sector_details['Theta / Delta']
#sector_portfolio_vega_delta_ratio = sector_details['Vega / Delta']
#sector_portfolio_delta_gamma_ratio = sector_details['Delta / Gamma']

sector_portfolio_theta_fig = px.bar(sector_portfolio_theta,title='sector portfolio theta',color_discrete_sequence=['red'])
sector_portfolio_delta_fig = px.bar(sector_portfolio_delta,title='sector portfolio delta',color_discrete_sequence=['blue'])
sector_portfolio_gamma_fig = px.bar(sector_portfolio_gamma,title='sector portfolio gamma',color_discrete_sequence=['green'])
sector_portfolio_vega_fig  = px.bar(sector_portfolio_vega,title='sector portfolio vega',color_discrete_sequence=['orange'])

#sector_portfolio_theta_delta_ratio_fig  = px.bar(sector_portfolio_theta_delta_ratio,title='sector portfolio Theta / Delta')
#sector_portfolio_vega_delta_ratio_fig  = px.bar(sector_portfolio_vega_delta_ratio,title='sector portfolio Vega / Delta')
#sector_portfolio_delta_gamma_ratio_fig  = px.bar(sector_portfolio_delta_gamma_ratio,title='sector portfolio Delta / Gamma')

portfolio_theta_fig = px.bar(portfolio_theta,title='portfolio theta',color_discrete_sequence=['red'])
portfolio_delta_fig = px.bar(portfolio_delta,title='portfolio delta',color_discrete_sequence=['blue'])
portfolio_gamma_fig = px.bar(portfolio_gamma,title='portfolio gamma',color_discrete_sequence=['green'])
portfolio_vega_fig  = px.bar(portfolio_vega,title='portfolio vega',color_discrete_sequence=['orange'])


portfolio_theta_delta_ratio_fig  = px.bar(portfolio_theta_delta_ratio,title='portfolio theta / delta')
portfolio_vega_delta_ratio_fig  = px.bar(portfolio_theta_delta_ratio,title='portfolio vega / delta')
portfolio_delta_gamma_ratio_fig  = px.bar(portfolio_delta_gamma_ratio,title='portfolio delta / gamma')

sector_portfolio_greeks_chart_layout = make_subplots(rows=2,cols=2,subplot_titles=("Sector Theta", 'Sector Delta', 'Sector Gamma', ' Sector Vega'))
sector_portfolio_greeks_chart_layout['layout'].update(height=800, title='Sector Greeks')

sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_theta_fig.data[0],row=1,col=1)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_delta_fig.data[0],row=1,col=2)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_gamma_fig.data[0],row=2,col=1)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_vega_fig.data[0],row=2,col=2)

sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_theta_fig.data[0],row=1,col=1)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_delta_fig.data[0],row=1,col=2)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_gamma_fig.data[0],row=2,col=1)
sector_portfolio_greeks_chart_layout.add_trace(sector_portfolio_vega_fig.data[0],row=2,col=2)

sector_portfolio_greeks_chart_layout.add_hline(y=sector_portfolio_theta.mean(), line_dash="dash", line_color="#000000", line_width=1,row=1,col=1)
sector_portfolio_greeks_chart_layout.add_hline(y=sector_portfolio_delta.mean(), line_dash="dash", line_color="#000000", line_width=1,row=1,col=2)
sector_portfolio_greeks_chart_layout.add_hline(y=sector_portfolio_gamma.mean(), line_dash="dash", line_color="#000000", line_width=1,row=2,col=1)
sector_portfolio_greeks_chart_layout.add_hline(y=sector_portfolio_vega.mean(), line_dash="dash", line_color="#000000", line_width=1,row=2,col=2)

sector_portfolio_greeks_chart_layout.update_layout(autosize=True)

sector_portfolio_greeks_chart_layout.show(config={'responsive': True})

portfolio_greeks_chart_layout = make_subplots(rows=2,cols=2,subplot_titles=("Portfolio Theta", 'Portfolio Delta', 'Portfolio Gamma', 'Portfolio Vega'))
portfolio_greeks_chart_layout['layout'].update(height=800, title='Portfolio Greeks')

portfolio_greeks_chart_layout.add_trace(portfolio_theta_fig.data[0],row=1,col=1)
portfolio_greeks_chart_layout.add_trace(portfolio_delta_fig.data[0],row=1,col=2)
portfolio_greeks_chart_layout.add_trace(portfolio_gamma_fig.data[0],row=2,col=1)
portfolio_greeks_chart_layout.add_trace(portfolio_vega_fig.data[0],row=2,col=2)

portfolio_greeks_chart_layout.add_hline(y=portfolio_theta.mean().iloc[0], line_dash="dash", line_color="#000000", line_width=1,row=1,col=1)
portfolio_greeks_chart_layout.add_hline(y=portfolio_delta.mean().iloc[0], line_dash="dash", line_color="#000000", line_width=1,row=1,col=2)
portfolio_greeks_chart_layout.add_hline(y=portfolio_gamma.mean().iloc[0], line_dash="dash", line_color="#000000", line_width=1,row=2,col=1)
portfolio_greeks_chart_layout.add_hline(y=portfolio_vega.mean().iloc[0], line_dash="dash", line_color="#000000", line_width=1,row=2,col=2)

portfolio_greeks_chart_layout.update_layout(autosize=True)
portfolio_greeks_chart_layout.show(config={'responsive': True})

#sector_portfolio_theta_delta_ratio_fig.update_layout(autosize=True)
#sector_portfolio_vega_delta_ratio_fig.update_layout(autosize=True)
#sector_portfolio_delta_gamma_ratio_fig.update_layout(autosize=True)

#portfolio_theta_fig.update_layout(autosize=True)
#portfolio_delta_fig.update_layout(autosize=True)
#portfolio_gamma_fig.update_layout(autosize=True)
#portfolio_vega_fig.update_layout(autosize=True)
#portfolio_theta_delta_ratio_fig.update_layout(autosize=True)
#portfolio_vega_delta_ratio_fig.update_layout(autosize=True)
#portfolio_delta_gamma_ratio_fig.update_layout(autosize=True)


#sector_portfolio_theta_delta_ratio_fig.show(config={'responsive': True})
#sector_portfolio_vega_delta_ratio_fig.show(config={'responsive': True})
#sector_portfolio_delta_gamma_ratio_fig.show(config={'responsive': True})


#portfolio_theta_delta_ratio_fig.show(config={'responsive': True})
#portfolio_vega_delta_ratio_fig.show(config={'responsive': True})
#portfolio_delta_gamma_ratio_fig.show(config={'responsive': True})



In [3]:
#Portfolio vs Market Returns 

average_return_50          = pm.rolling(algorithm.percent_return,50).mean(axis=1)[-200:]
weighted_average_sharpe_50 = pm.weighted_average_rolling(algorithm.percent_return,50)[-200:]
market_return_50 = comp.rolling(algorithm.percent_return,50,benchmark['Close'])[-200:]

df = pd.DataFrame({
    'Average Portfolio Return' : average_return_50,
    'Weighted Average Portfolio Return': weighted_average_sharpe_50,
    'Average Market Return' : market_return_50 
})

average_return = px.line(df,title='Portfolio Returns')
average_return.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
average_return.update_layout(autosize=True)
average_return.show(config={'responsive': True})

average_sharpe_50          = pm.rolling(algorithm.sharpe,50,risk_free_rate['Close']).mean(axis=1)[-200:]
weighted_average_sharpe_50 = pm.weighted_average_rolling(algorithm.sharpe,50,risk_free_rate['Close'])[-200:]
market_sharpe_50 = comp.rolling(algorithm.sharpe,50,benchmark['Close'],risk_free_rate['Close'])[-200:]

df = pd.DataFrame({
    'Average Portfolio Sharpe' : average_sharpe_50,
    'Weighted Average Portfolio Sharpe': weighted_average_sharpe_50,
    'Average Market Sharpe' : market_sharpe_50 
})

average_sharpe = px.line(df,title='Portfolio Sharpe' )
average_sharpe.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
average_sharpe.update_layout(autosize=True)
average_sharpe.show(config={'responsive': True})





In [4]:
#Portfolio vs Market return distribution plots
average_return_50          = pm.retrieve_ticker_data().pct_change().mean(axis=1).dropna()[-200:]
market_return_50           = benchmark['Close'].pct_change().dropna()[-200:]

hist_data     = [average_return_50.tolist(), market_return_50.tolist()]
group_labels = ['Average Portfolio Return Distribution', 'Market Return Distribution']


fig = ff.create_distplot(hist_data=hist_data,group_labels=group_labels,bin_size=[.005,.005])
fig.show()

In [5]:
#Portfolio Asset Sharpe Comparison

height= 500
width = 1400

sharpe_50 = pm.rolling(algorithm.sharpe,50,risk_free_rate['Close'])
top_num = 10

last_row_50D = sharpe_50.iloc[-1].sort_values()

top_50D       = list(last_row_50D[-top_num:].index)
bottom_50D    = list(last_row_50D[:top_num].index)

top_performance_chart_50 = px.line(sharpe_50[top_50D][-100:],title='Top 50 Day Portfolio Sharpe')
top_performance_chart_50.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)

bottom_performance_chart_50 = px.line(sharpe_50[bottom_50D][-100:],title='Bottom 50 Day Portfolio Sharpe')
bottom_performance_chart_50.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)

top_performance_chart_50.update_layout(autosize=True)
top_performance_chart_50.show(config={'responsive': True})

bottom_performance_chart_50.update_layout(autosize=True)
bottom_performance_chart_50.show(config={'responsive': True})



In [6]:
#Average Correlation with stock market

broad_markets = {
    'SPY' : 'long'
}
height= 600
width = 1400

time_frames = [50,200]

broad_market_portfolio = Portfolio(universe=universe)
broad_market_portfolio.add_tickers(broad_markets)

pm.load(period='3y', interval='1d')
broad_market_portfolio.load(period='3y', interval='1d')

broad_market_data = broad_market_portfolio.retrieve_ticker_data()
portfolio_data    = pm.retrieve_ticker_data()

stock_market_corr = pm.rolling_multi_timeframe(algorithm.correlation, time_frames, broad_market_data['SPY'])

broad_market_portfolio.retrieve_assets().keys

def average_portfolio_correlation(market_df):
    average_window_df = pd.DataFrame()
    for window in time_frames:
        df = pd.DataFrame()
        for key in pm.retrieve_assets().keys():
            df[str(key) + '_' + str(window)]=market_df[key][window]
        average_window_df['Average_' + str(window)] = df.mean(axis=1)
    return average_window_df

stock_market_corr_chart         = px.line(average_portfolio_correlation(stock_market_corr),title='Stock market Correlation')
stock_market_corr_chart.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
stock_market_corr_chart.update_layout(autosize=True)
stock_market_corr_chart.show(config={'responsive': True})

In [7]:
#Average Portfolio correlation of each individual assets with other portfolio components
height= 500
width = 1400


pm.load(period='1y')
fifty_d= pm.retrieve_ticker_data()[-50:]

pm.load(period='1y')
two_hundred_d = pm.retrieve_ticker_data()[-200:]

corr_50D_matrix   = comp.create_correlation_heatmap(fifty_d)

scalar_multiplier = .7

fifty_d_corr       = fifty_d .corr().round(2).mean(axis=0).sort_values(ascending=False)

#s1, s2 = fifty_d_corr.align(two_hundred_d_corr)
#difference_corr = ((s1 + s2) / 2).sort_values(ascending=False)

average_corr = px.bar(fifty_d_corr,title='50 Day Portfolio correlation')
average_corr.add_hline(y=0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
average_corr.add_hline(y=-0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)



average_corr.update_layout(autosize=True)
average_corr.show(config={'responsive': True})

In [8]:
#Portfolio correlation heat map on multiple time frames 
height= 1200
width = 1200

fig_layout = make_subplots(rows=1,
                    cols=1,
                    subplot_titles=("50 Day Correlation"),
                    vertical_spacing = 0.05)

fig_layout.update_layout(title_text="Portfolio Correlation")

scalar_multiplier = .7
fig_layout['layout'].update(width=width* scalar_multiplier,height=height* scalar_multiplier, title='')

ticker_data = pm.retrieve_ticker_data()

returns_50 = ticker_data[-50:]


returns_50_fig  = plot.create_correlation_figure(returns_50)

fig_layout.add_trace(returns_50_fig.data[0], col=1,row=1)
fig_layout.update_layout(title_text="Portfolio Correlation")
fig_layout.update_layout(autosize=True)
fig_layout.show(config={'responsive': True})


In [9]:
#Multi-time frame 'Rank' of Highest, Lowest, and Least correlated Assets to portfolio

''' 
1. Retrieve all assets in portfolio 
2. Retrieve all assets in universe to compare
3. Calculate average correlation of each portfolio asset to each asset in universe 21,50,200 window
4. Rank each asset in universe by level of correlation by Highest, Lowest, least for each window
5. find the average rank for all windows for each asset and sort by average rank
6. display sorted table displaying each windows rank and average rank
'''

pm.load(period='3y',interval='1d')
portfolio_assets = pm.retrieve_ticker_data()



In [10]:
#Options Portfolio

import yfinance as yf
msft = yf.Ticker("MSFT")
msft.options

for expiration in msft.options:
    print(msft.option_chain(expiration).calls)

# The usual py_vollib syntax

import numpy as np
import pandas as pd
import yfinance as yf
from Quantapp.Algorithms import Algorithms
data = yf.Ticker('SPY')
algorithms = Algorithms()

from py_vollib.black_scholes.greeks.analytical import *
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes import *



option_chain = data.option_chain()

strike_list = option_chain.calls['strike']
current_price = data.history(period='5y', interval='1D').iloc[-1]['Close']
volatility = algorithms.rolling(algorithms.standard_deviation, 200, data.history(period='1Y', interval='1D')['Close'])
sigma= volatility.iloc[-1] 
sigma /=10
sigma /=10
flag = 'c'  # 'c' for call, 'p' for put
F = 100  # Underlying asset price
K = 90  # Strike
t = 0.5  # (Annualized) time-to-expiration
r = 0.01  # Interest free rate

delta_list = []
gamma_list = []
theta_list = []
vega_list  = []
rho_list  = []
implied_volatility_list = []
price_list = []

for strike in strike_list:
    delta_list.append(delta(flag, current_price, strike, t, r, sigma))
    gamma_list.append(gamma(flag, current_price, strike, t, r, sigma))
    theta_list.append(theta(flag, current_price, strike, t, r, sigma))
    vega_list.append(vega(flag, current_price, strike, t, r, sigma))
    rho_list.append(rho(flag, current_price, strike, t, r, sigma))
    price = black_scholes(flag, current_price, strike, t, r, sigma)
    price_list.append(price)
    implied_volatility_list.append(iv(price, current_price, strike, t, r, flag))
  
greeks = {
    'strike': strike_list,
    'delta': delta_list,
    'gamma': gamma_list,
    'theta': theta_list,
    'vega' : vega_list,
    'rho': rho_list,
    'IV': implied_volatility_list,
    'price': price_list,
}


df = pd.DataFrame(greeks)
df.index = strike_list

df
#gamma(flag, F, K, t, r, sigma)

         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   MSFT230414C00170000 2023-03-29 14:09:13+00:00   170.0     110.13  111.80   
1   MSFT230414C00175000 2023-03-29 14:09:13+00:00   175.0     105.12  106.90   
2   MSFT230414C00180000 2023-04-05 19:48:40+00:00   180.0     104.67  102.05   
3   MSFT230414C00200000 2023-03-27 14:58:58+00:00   200.0      79.00   81.85   
4   MSFT230414C00210000 2023-04-04 15:10:22+00:00   210.0      78.41   71.95   
5   MSFT230414C00215000 2023-03-31 19:55:30+00:00   215.0      75.55   66.65   
6   MSFT230414C00220000 2023-04-06 17:23:20+00:00   220.0      68.79   62.05   
7   MSFT230414C00225000 2023-03-07 18:19:33+00:00   225.0      32.45   64.75   
8   MSFT230414C00230000 2023-04-10 18:43:38+00:00   230.0      58.63   51.95   
9   MSFT230414C00235000 2023-04-11 15:32:20+00:00   235.0      47.55   47.20   
10  MSFT230414C00240000 2023-04-11 15:13:37+00:00   240.0      42.44   42.10   
11  MSFT230414C00242500 2023-04-06 18:29

ModuleNotFoundError: No module named 'Quantapp.Algorithms'